### 7 순환신경망

#### 7.1 기본 순환 신경망

##### 7.1.5 기본 순환 신경망 구현

In [2]:
# 라이브러리 로드
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
# 데이터 로드
df = pd.read_csv("./data/kospi.csv")
scaler = MinMaxScaler()
# min max 스케일러를 사용하여 데이터를 0~1 사이의 값으로 보정해준다.
df[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-30,0.722898,0.732351,0.745525,0.759235,2206.199951,0.242113
1,2019-01-31,0.763058,0.750069,0.769089,0.757866,2204.850098,0.274771
2,2019-02-01,0.751894,0.745714,0.769280,0.756456,2203.459961,0.241609
3,2019-02-07,0.755809,0.742538,0.764596,0.756415,2203.419922,0.215603
4,2019-02-08,0.731584,0.717777,0.739548,0.729669,2177.050049,0.197057


In [5]:
# 텐서 데이터 만들기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

inputs = df[['Open', 'High', 'Low', 'Volume']].values
targets = df[['Close']].values

In [6]:
# 시퀀스 데이터 만들기
from msilib import sequence


def seq_data(x, y, seq_length):
    # 데이터를 일정 길이 기준으로 잘라서 배열로 만들어 반환
    # ex) x:[1,2,3,4,5], y:[1,2,3,4,5], seq_length=3  경우
    # 배열의 첫번째 내용:  x_seq -> [1,2,3], y_seq -> 4
    # 배열의 두번째 내용:  x_seq -> [2,3,4], y_seq -> 5
    x_seq=[]
    y_seq=[]
    len = len(x)
    for i in range(len-seq_length):        
        x_seq.append(x[i: i+seq_length])
        y_seq.append(y[i+seq_length])
    return torch.FloatTensor(x_seq.to(device)), torch.FloatTensor(y_seq.to(device)).view(-1, 1)
    # y_seq를 view(-1,1) 로 2차원배열로 만든 것은 오차함수로 사용할 MSE Loss가 기본적으로 이차원 함수를 요구하기 때문